In [8]:
import pandas as pd
import pickle
from os import listdir
import numpy as np
from scipy import stats
from kneed import KneeLocator

def ros_stop(res, filename, target=50): 
    #print(filename, target)
    no_impv = target
    old_val = -1
    for i, x in enumerate(res['supervised']['pos']):
        if old_val != x:
            old_val = x
            no_impv = target
        else:
            no_impv -= 1
        if no_impv == 0:
            break
            
    cost = res['supervised']['x'][i]/res['true'][1] #int(round(res['supervised']['x'][i]/res['true'][1]*100,0))
    recall = res['supervised']['pos'][i]/res['true'][0] #int(round(res['supervised']['pos'][i]/res['true'][0]*100,0))
    print('ros' + str(target) + ": " + str(int(round(100*recall,0))) + " (" + str(int(round(100*cost,0))) + ")" )
    return cost, recall

    
def cormack_stop(results, filename, rho=6):
    #print(filename, rho)
    for i in range(5, len(results['supervised']['pos'])):
        y = results['supervised']['pos'][:i]
        x = results['supervised']['x'][:i]
        kneedle = KneeLocator(x, y, S=1.0, curve='concave', direction='increasing')
        knee = kneedle.knee
        if knee is None or knee==0:
            continue
        x_less = results['supervised']['x'][:int(knee/10)]
        y_less = results['supervised']['pos'][:int(knee/10)]
        x_more = results['supervised']['x'][int(knee/10)+1:i]
        y_more = results['supervised']['pos'][int(knee/10)+1:i]
        slope_less, intercept, r_value, p_value, std_err = stats.linregress(x_less, y_less)
        slope_more, intercept, r_value, p_value, std_err = stats.linregress(x_more, y_more)
        if slope_more == 0:
            continue
        if rho < slope_less/slope_more:
            cost = results['supervised']['x'][i]/results['true'][1] #int(round(results['supervised']['x'][i]/results['true'][1] * 100,0))
            recall = results['supervised']['pos'][i]/results['true'][0] #int(round(results['supervised']['pos'][i]/results['true'][0] * 100,0))
            print('cormack' + str(rho) + ': ' + str(int(round(100*recall,0))) + " (" + str(int(round(100*cost,0))) + ")" )
            #print(int(knee/10), round(results['supervised']['pos'][int(knee/10)]/results['true'][0] * 100,2),
                 #round(results['supervised']['x'][int(knee/10)]/results['true'][1] * 100,2))
            return cost, recall
    return 1, 1


files = listdir('../../dump_90_boost_dt_combined')
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()

for file in files:
    with open("../../dump_90_boost_dt_combined/"+file,"rb") as handle:
        results = pickle.load(handle)
        #if 'hibernate' not in file:
        #    continue
        print(file)
        df1[file] = ros_stop(results, file, 50)
        df2[file] = ros_stop(results, file, 10)
        #ros_stop(results, file, 100)
        #print(file, 10)
        #ros_stop(results, file, 10)
        df3[file] = cormack_stop(results, file, 6)
        df4[file] = cormack_stop(results, file, 12)
        
print('ros50', df1.T.describe().loc['75%'] - df1.T.describe().loc['25%'])
print('ros10', df2.T.describe().loc['75%'] - df2.T.describe().loc['25%'])
print('cor6', df3.T.describe().loc['75%'] - df3.T.describe().loc['25%'])
print('cor12', df4.T.describe().loc['75%'] - df4.T.describe().loc['25%'])
df = pd.concat([df1, df2, df3, df4])
df = df.T
df.columns = ['ros@50cost', 'ros@50recall', 'ros@10cost', 'ros@10recall', 'cormack@6cost', 'cormack@6recall', 'cormack@12cost', 'cormack@12recall']
df

def get_score_str(col_name, df):
    col = df.loc[col_name].tolist()
    str1 = col_name + '\n'
    str1 += ' '.join(str(e) for e in col)
    str1 += '\n\n'
    return str1

df = df.T
score_str = get_score_str('ros@50recall', df)
score_str += get_score_str('ros@10recall', df)
score_str += get_score_str('cormack@6recall', df)
score_str += get_score_str('cormack@12recall', df)
df.T.describe()


apache-ant-1.7.0.pkl
ros50: 72 (28)
ros10: 58 (6)
cormack6: 58 (5)
cormack12: 58 (6)
apache-jmeter-2.10.pkl
ros50: 87 (18)
ros10: 81 (7)
cormack6: 77 (5)
cormack12: 81 (8)
argouml.pkl
ros50: 96 (33)
ros10: 89 (19)


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:116: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:126: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:128: RuntimeWarning: invalid value enco

cormack6: 89 (18)
cormack12: 89 (20)
columba-1.4-src.pkl
ros50: 93 (13)
ros10: 93 (7)
cormack6: 90 (4)
cormack12: 90 (4)
emf-2.4.1.pkl
ros50: 63 (28)
ros10: 47 (6)
cormack6: 42 (2)
cormack12: 47 (5)
hibernate-distribution-3.3.2.GA.pkl
ros50: 85 (40)
ros10: 85 (27)
cormack6: 77 (17)
cormack12: 78 (20)
jEdit-4.2.pkl
ros50: 53 (9)
ros10: 53 (6)
cormack6: 33 (2)


C:\ProgramData\Anaconda3\lib\site-packages\kneed\knee_locator.py:115: RuntimeWarning: No local maxima found in the distance curve
The line is probably not polynomial, try plotting
the distance curve with plt.plot(knee.xd, knee.yd)
Also check that you aren't mistakenly setting the curve argument
  "Also check that you aren't mistakenly setting the curve argument", RuntimeWarning)


cormack12: 53 (96)
jfreechart-1.0.19.pkl
ros50: 86 (24)
ros10: 54 (7)
cormack6: 54 (5)
cormack12: 54 (5)
jruby-1.4.0.pkl
ros50: 96 (33)
ros10: 93 (16)
cormack6: 93 (15)
cormack12: 93 (17)
sql12.pkl
ros50: 73 (25)
ros10: 60 (6)
cormack6: 55 (3)
cormack12: 60 (6)
ros50 0    0.117345
1    0.191239
dtype: float64
ros10 0    0.077311
1    0.335826
dtype: float64
cor6 0    0.097342
1    0.325353
dtype: float64
cor12 0    0.133563
1    0.325764
dtype: float64


,ros@50cost,ros@50recall,ros@10cost,ros@10recall,cormack@6cost,cormack@6recall,cormack@12cost,cormack@12recall
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.251316,0.803611,0.106867,0.711415,0.075520,0.667498,0.185781,0.702943
std,0.094336,0.145840,0.073407,0.184185,0.063425,0.213697,0.277476,0.176891
min,0.094943,0.527344,0.056191,0.471154,0.019376,0.328125,0.041744,0.471154
25%,0.197886,0.720860,0.061989,0.546952,0.029413,0.539152,0.054862,0.546952
50%,0.261226,0.855017,0.068816,0.702900,0.049157,0.673766,0.067779,0.690715
75%,0.315232,0.912099,0.139300,0.882778,0.126756,0.864505,0.188426,0.872716
max,0.404313,0.962137,0.269542,0.926471,0.179856,0.926045,0.955241,0.926045


In [ ]:
def get_stat_str(col_name, df):
    col = df.loc[col_name].tolist()
    str1 = col_name + '\n'
    str1 += ' '.join(str(e) for e in col)
    str1 += '\n\n'
    return str1
mystr = get_stat_str('ros@50cost', df)
mystr += get_stat_str('ros@10cost', df)
mystr += get_stat_str('cormack@6cost', df)
mystr += get_stat_str('cormack@12cost', df)
mystr

with open ('statRQ3Cost.txt', 'w') as f:
    f.write(mystr)
    